In [1]:
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow import keras

In [2]:
# split the mnist data into train and test
(train_img,train_label),(test_img,test_label) = keras.datasets.mnist.load_data()
 
# reshape and scale the data
train_img = train_img.reshape([-1, 28, 28, 1])
test_img = test_img.reshape([-1, 28, 28, 1])
train_img = train_img/255.0
test_img = test_img/255.0
 
# convert class vectors to binary class matrices --> one-hot encoding
train_label = keras.utils.to_categorical(train_label)
test_label = keras.utils.to_categorical(test_label)

In [3]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (5, 5), padding="same", input_shape=[28, 28, 1]),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64, (5, 5), padding="same"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
model.fit(train_img,train_label, validation_data=(test_img,test_label), epochs=15)
test_loss,test_acc = model.evaluate(test_img, test_label)
print('Test accuracy:', test_acc)

Epoch 1/15
1875/1875 [==============================] - 127s 68ms/step - loss: 0.1189 - accuracy: 0.9632 - val_loss: 0.0774 - val_accuracy: 0.9751
Epoch 2/15
1875/1875 [==============================] - 133s 71ms/step - loss: 0.0555 - accuracy: 0.9832 - val_loss: 0.0500 - val_accuracy: 0.9845
Epoch 3/15
1875/1875 [==============================] - 132s 70ms/step - loss: 0.0439 - accuracy: 0.9877 - val_loss: 0.0496 - val_accuracy: 0.9885
Epoch 4/15
1875/1875 [==============================] - 137s 73ms/step - loss: 0.0401 - accuracy: 0.9887 - val_loss: 0.0433 - val_accuracy: 0.9896
Epoch 5/15
1875/1875 [==============================] - 128s 68ms/step - loss: 0.0353 - accuracy: 0.9908 - val_loss: 0.0541 - val_accuracy: 0.9882
Epoch 6/15
1875/1875 [==============================] - 131s 70ms/step - loss: 0.0349 - accuracy: 0.9914 - val_loss: 0.0608 - val_accuracy: 0.9869
Epoch 7/15
1875/1875 [==============================] - 129s 69ms/step - loss: 0.0311 - accuracy: 0.9928 - val_loss: 0

In [10]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

#model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
        
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        a,b,c,d = rect
        rect=(a+4,b+4,c-4,d-4)
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
       
app = App()
mainloop()

1/1 [==============================] - 0s 18ms/step
